# Exercise 1.2 Linear Regression via Normal Equations

 Importing the required libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg
import random
import math
import warnings
warnings.filterwarnings('ignore')


### Function Name: Create_data
### Parameter: Filename
This function reads the data, shuffles them and split it to input variables and target values

After analysing the Mean Square error with differernt input variables, price,income,gasolines and pumps are taken into account. Also, the variables namely CarWash, Competitors, Zipcode, Address, ID, Restaurant does not play a major role as it can be seen in the data file. And when we analyse the top 10 incomes, we get to see that location of the station i.e., highways, intersection influences the income but as the error increases when we use them, they are also neglected

In [2]:
def create_data(filename):
    Data = pd.read_csv(filename)
    Req_Data = Data.drop(columns = ['Unnamed: 0','ID','Restaurant',
                                    'CarWash','Competitors','Zipcode','Address'
                                     ,'Highway','IntersectionStoplight','Name','Stoplight'
                                   ,'Interior','Intersection','Brand'])
    Shuffle_data = Req_Data.sample(frac = 1)
    Shuffle_data = pd.get_dummies(Shuffle_data)
    Y_data = Shuffle_data.Income/np.linalg.norm(Shuffle_data.Income)
    #Y_data = Shuffle_data.Income
    X_data = Shuffle_data.drop(columns = ['Income'])
    X_data = X_data.values
    return X_data,Y_data



###  

###  

###   

### Function Name: create_Test_Train_data
### Parameter: Data of input variables and target
This function splits the input variables and target data to Train and Test data with 80% and 20% of the data respectively

In [3]:
def create_Test_Train_data(X_data,Y_data):
    Y_train = Y_data[:math.ceil(0.8*len(Y_data))]
    Y_test = Y_data[math.ceil(0.8*len(Y_data)):]
    X_train = X_data[:math.ceil(0.8*len(X_data))]
    X_test = X_data[math.ceil(0.8*len(X_data)):]
    X = np.ones((X_train.shape[0],X_train.shape[1]+1))
    X[:,1:] = X_train
    A = np.dot(X.T,X)
    B = np.dot(X.T,Y_train)
    return Y_train,Y_test,X_train,X_test,A,B

### Function Name: slv_beta    
### Parameter: Input variables matrix and Target vector
This function returns the set of parameters trained from the input variables

In [4]:
def slv_beta(A,B):
    b = np.linalg.solve(A,B)
    return b

### Function Name: Predict
### Parameter: Parameter vector
This function predicts the target value for the test data and returns the same

In [ ]:
def Predict(b):
    Test = np.ones((X_test.shape[0],X_test.shape[1]+1))
    Test[:,1:] = X_test
    Pred_Y_test = np.matmul(Test,b)
    return Pred_Y_test

##  

### Function Name: MSE_Plot
### Parameter: Predicted vector
This function calculates the Root Mean Square Error for the data predicted by the model

In [ ]:
def RMSE_Plot(Pred_Y_test):
    total = 0
    for i in range(0,len(Y_test)):
        total+=pow(Pred_Y_test[i]-Y_test.values[i],2)
    print("\nMSE:     ",total/len(Y_test))
    plt.plot(Y_test.values,"bo")
    plt.plot(Pred_Y_test,"ro")
    plt.xlabel("Value")
    plt.ylabel("Test and Prediction")
    plt.show()
    return RMSE

### Function Name: cholesky_matrix
### Parameter: Input variables matrix
This function does operations within the matrix and returns the Lower Triangular Matrix to perform backward substitution process for getting the set of parameters

In [ ]:
def cholesky_matrix(A):
    
    if A.shape[0] == A.shape[1]:
        A = A.astype(float)
        L = np.zeros_like(A)
        size = np.size(A, 0)
        L[0, 0] = pow(A[0, 0],0.5)
        for i in range(1, size):
            L[i:, i-1] = (A[i:, i-1] - L[i:, :i-1].dot(L[i-1, :i-1])) / L[i-1, i-1]
            L[i, i] = pow((A[i, i] - L[i, :i].dot(L[i, :i])),0.5)
        return L
    
    else:
        print ("Invalid Matrix")

### Function Name: gauss_beta
### Parameter: Input variables matrix and Target vector

This function solves the set of linear equation returns the set of parameters trained from the input variables using Gaussian elimination method

In [ ]:
def gauss_beta(A,B):
    if A.shape[0] == A.shape[1]:
        Gauss_mat = np.ones((A.shape[0],A.shape[1]+1))
        Gauss_mat[:,:A.shape[0]] = A[:,:A.shape[0]]
        Gauss_mat[:,A.shape[0]] = B
        A = Gauss_mat
        n = len(A)
        for i in range(0, n):
            # Search for maximum in this column
            maxEl = abs(A[i][i])
            maxRow = i
            for k in range(i+1, n):
                if abs(A[k][i]) > maxEl:
                    maxEl = abs(A[k][i])
                    maxRow = k
            # Swap maximum row with current row (column by column)
            for k in range(i, n+1):
                tmp = A[maxRow][k]
                A[maxRow][k] = A[i][k]
                A[i][k] = tmp

            # Make all rows below this one 0 in current column
            for k in range(i+1, n):
                c = -A[k][i]/A[i][i]
                for j in range(i, n+1):
                    if i == j:
                        A[k][j] = 0
                    else:
                        A[k][j] += c * A[i][j]

        # Solve equation Ax=b for an upper triangular matrix A
        x = [0 for i in range(n)]
        for i in range(n-1, -1, -1):
            x[i] = A[i][n]/A[i][i]
            for k in range(i-1, -1, -1):
                A[k][n] -= A[k][i] * x[i]
        return x
    
    else:
        print ("Invalid Matrix")

### Function Name: forward_substitution and backward_substitution
### Parameter: Input variables matrix and Target vector
These functions performs the forward and backward substitutions for the lower triangle and upper triangle matrix respectively

In [ ]:
def forward_substitution(A, B):
    
    if A.shape[0] == A.shape[1]:
        x = np.zeros_like(B)
        for i in range(np.size(x, 0)):
            x[i] = (B[i] - A[i, :i].dot(x[:i])) / A[i, i]
        return x

    else:
        print ("Invalid Matrix")
        
def backward_substitution(A, B):
    
    if A.shape[0] == A.shape[1]:
        x = np.zeros_like(B)
        for i in range(np.size(x, 0)).__reversed__():
            x[i] = (B[i] - A[i, i+1:].dot(x[i+1:])) / A[i, i]
        return x
    
    else:
        print ("Invalid Matrix")

### Function Name: QR_beta
### Parameter: Input variables matrix and Target vector

This function solves the set of linear equation returns the set of parameters trained from the input variables using QR decomposition method

In [ ]:
def QR_beta(A,B):
    Q,R = np.linalg.qr(A)
    dot_QB = np.dot(Q.T,B)
    b = backward_substitution(R,dot_QB)
    return b

### Function Name: linalg_slv
### Parameter: Input variables matrix and Target vector

This function solves the set of linear equation using normal equations. From the learned set of parameters, the target data is predicted from the model and the error is computed

In [ ]:
def linalg_slv(A,B):
    if A.shape[0] == A.shape[1]:
        b = slv_beta(A,B)
        print("Solving linear equations using Normal equations:\n")
        print("Set of Parameters:\n",b)
        Pred_Y_test = Predict(b)
        RMSE = RMSE_Plot(Pred_Y_test)
        return Pred_Y_test
        
    else:
        print ("Invalid Matrix")

### Function Name: gaussian_elimination
### Parameter: Input variables matrix and Target vector

This function solves the set of linear equation using Gaussian elimination method. From the learned set of parameters, the target data is predicted from the model and the error is computed

In [ ]:
def gaussian_elimination(A,B):
    b = gauss_beta(A,B)
    print("Solving linear equations using Gaussian elimination:\n")
    print("Set of Parameters:\n",b)
    Pred_Y_test = Predict(b)
    RMSE = RMSE_Plot(Pred_Y_test)
    return Pred_Y_test

### Function Name: cholesky_decomposition
### Parameter: Input variables matrix and Target vector

This function solves the set of linear equation using Cholesky decomposition method. From the learned set of parameters, the target data is predicted from the model and the error is computed

In [ ]:
def cholesky_decomposition(A,B):
    mat_cholesky= cholesky_matrix(A)
    y = forward_substitution(mat_cholesky,B)
    y=np.array(y)
    b = backward_substitution(mat_cholesky.T,y)
    print("Solving linear equations using Cholesky Decomposition:\n")
    print("Set of Parameters: \n",b)
    Pred_Y_test = Predict(b)
    RMSE = RMSE_Plot(Pred_Y_test)
    return Pred_Y_test

### Function Name: QR_decomposition
### Parameter: Input variables matrix and Target vector

This function solves the set of linear equation using QR decomposition method. From the learned set of parameters, the target data is predicted from the model and the error is computed

In [ ]:
def QR_decomposition(A,B):
    b = QR_beta(A,B)
    print("Solving linear equations using QR Decomposition:\n")
    print("Set of Parameters:\n",b)
    Pred_Y_test = Predict(b)
    RMSE = RMSE_Plot(Pred_Y_test)
    return Pred_Y_test

In [ ]:
def RMSE(Predicted,Y_test):
    return np.sqrt(np.sum(pow(Y_test - np.mean(Predicted),2))/len(Y_test))

### Main function to call other functions

In [ ]:
if __name__ == "__main__":
    
    filename = "GasPrices.csv"
    X_data,Y_data = create_data(filename)
    Y_train,Y_test,X_train,X_test,A,B = create_Test_Train_data(X_data,Y_data)

In [ ]:
Pred_slv = linalg_slv(A,B)    
residual_slv  = abs(Y_test.values - np.mean(Pred_slv))
print("\nAverage Residual: ",np.average(residual_slv))
print("\nRMSE : ",round(RMSE(residual_slv,Y_test),5))
plt.plot(residual_slv ,Y_test,'ro')
plt.xlabel("Residual")
plt.ylabel("Test data")
plt.title("Residual vs Test data (Normal Equations)")
plt.show()

In [ ]:
Pred_ge = gaussian_elimination(A,B)
residual_ge = abs(Y_test.values - np.mean(Pred_ge))
print("\nAverage Residual: ",np.average(residual_ge))
print("\nRMSE : ",round(RMSE(residual_ge,Y_test),5))
plt.plot(residual_ge ,Y_test,'bo')
plt.xlabel("Residual")
plt.ylabel("Test data")
plt.title("Residual vs Test data (Gaussian Elimination)")
plt.show()

In [ ]:
Pred_qr = QR_decomposition(A,B)
residual_qr = abs(Y_test.values - np.mean(Pred_qr))
print("\nAverage Residual: ",np.average(residual_qr))
print("\nRMSE : ",round(RMSE(residual_qr,Y_test),5))
plt.plot(residual_qr ,Y_test,'go')
plt.xlabel("Residual")
plt.ylabel("Test data")
plt.title("Residual vs Test data (QR Decomposition)")
plt.show()

In [ ]:
Pred_cholesky = cholesky_decomposition(A,B)
residual_cholesky = abs(Y_test.values - np.mean(Pred_cholesky))
print("\nAverage Residual: ",np.average(residual_cholesky))
print("\nRMSE : ",round(RMSE(residual_cholesky,Y_test),5))
plt.plot(residual_cholesky ,Y_test,'yo')
plt.xlabel("Residual")
plt.ylabel("Test data")
plt.title("Residual vs Test data (Cholesky Decomposition)")
plt.show()